In [9]:
import sympy
sympy.init_printing()

In [12]:
t, dt = sympy.symbols('t, \\delta{t}')
t, dt

(t, \delta{t})

In [17]:
ds1 = sympy.sech(t + dt) - sympy.sech(t)
ds2 = ((t + dt) - sympy.tanh(t + dt)) - (t - sympy.tanh(t))
ds1, ds2
ds = ds1**2 + ds2**2
ds.expand()

         2                                                               2    
\delta{t}  + 2⋅\delta{t}⋅tanh(t) - 2⋅\delta{t}⋅tanh(\delta{t} + t) + tanh (t) 

                                      2                      2                
- 2⋅tanh(t)⋅tanh(\delta{t} + t) + tanh (\delta{t} + t) + sech (t) - 2⋅sech(t)⋅

                          2               
sech(\delta{t} + t) + sech (\delta{t} + t)

In [15]:
def Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    Mathematical formulation reference:
    https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
    :param function_expression: Sympy expression of the function
    :param variable_list: list. All variables to be approximated (to be "Taylorized")
    :param evaluation_point: list. Coordinates, where the function will be expressed
    :param degree: int. Total degree of the Taylor polynomial
    :return: Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial

In [22]:
deg = 2
ll = Taylor_polynomial_sympy(function_expression=ds.expand(), # 计算 ds^2
                             variable_list=[dt],
                             evaluation_point=[0],
                             degree=deg)
ll.simplify()

         2     2   
\delta{t} ⋅tanh (t)